# Getting Started with Global Land Cover Mapping and Estimation Yearly 30 meter (GLanCE30)

## Summary  

This tutorial was developed to examine changes in Land Cover surrounding the [Waubay National Wildlife Refuge](https://www.fws.gov/refuge/waubay) in northeast South Dakota (USA). **The goal of the project is to demonstrate a practical use case of using GLanCE30 to observe Land Cover change in this area over time without downloading the GLanCE30 source data.** In this notebook we will show an Land Cover timeseries from GLanCE30 data in the Cloud using the `earthaccess` and `rioxarray` libraries. This tutorial will show how to find the GLanCE30 data available in the cloud for our specific time period, bands (layers), and region of interest. After finding the desired data, we will load subsets of the cloud optimized geotiffs (COGs) into a Jupyter Notebook directly from the cloud, and show Land Cover change in tabular, quantitative visualization and map chart formats.

## Background  

NASA’s Making Earth System Data Records for Use in Research Environments ([MEaSUREs](https://earthdata.nasa.gov/about/competitive-programs/measures?_gl=1*xc5qp5*_ga*OTg2MDk1NDkuMTc0Njc5NjQ1MQ..*_ga_0YWDZEJ295*czE3NDcxNTgxMTYkbzQkZzEkdDE3NDcxNjAxNTYkajAkbDAkaDA.)) Global Land Cover Mapping and Estimation Yearly 30 meter ([GLanCE30](https://doi.org/10.5067/MEaSUREs/GLanCE/GLanCE30.001)) project produces an annual 30 meter global land cover and land cover change data product derived from the Landsat 5 Thematic Mapper (TM), Landsat 7 Enhanced Thematic Mapper Plus (ETM+), and Landsat 8 Operational Land Imager (OLI). GLAnce science data layers are designed to describe two landscape attributes:

1.	Land Cover and Land Cover Change. Four SDSs provide information related to: (1) the land cover class; (2) the estimated quality associated with the land cover class1; (3) the previous land cover class for those pixels where change occured; and (4) the approximate day of year (DOY) of change.
2.	Magnitude, Seasonality, and Changes in Greenness. Four SDSs are included that characterize annual greenness at each pixel via the Enhanced Vegetation Index (EVI2): (1) median; (2) amplitude; (3) rate of change (if present); and (4) magnitude of change in EVI2 median for those pixels where change occurred.

GLanCE products are useful for a variety of applications including monitoring the response of terrestrial ecosystems and land management. 

NASA's Land Processes Distributed Active Archive Center (LP DAAC) archives and distributes GLanCE30 products in the LP DAAC Cumulus cloud archive as Cloud Optimized GeoTIFFs (COG). This tutorial will demonstrate because these data are stored as COGs, this tutorial will teach users how to load subsets of individual files into memory for just the layers you are interested in. This tutorial covers how to search for, process and "stack" the GLanCE30 data over a region of interest into an [xarray](http://xarray.pydata.org/en/stable/) data array filter and visualize land cover and land change over the time series. 

## Requirements  

- A [NASA Earthdata Login](https://urs.earthdata.nasa.gov/) account is required to download the data used in this tutorial. You can create an account at the link provided.

- A compatible Python environment with the VSWIR Imaging and Thermal Applications, Learning, and Science ([VITALS](https://github.com/nasa/VITALS/tree/main)) environment installed. Please refer to the [Setup](https://github.com/nasa/VITALS/blob/main/setup/setup_instructions.md) Instructions for setting up the VITALS Environment.

## Learning Objectives

- How to work with GLanCE30 ([GLANCE30v001](https://doi.org/10.5067/MEaSUREs/GLanCE/GLanCE30.001)) data products  
- How to query and GLanCE30 HLS data using the earthaccess library  
- How to access and work with GLanCE30 data

## Data Used  

Annual 30 meter (m) Global Land Cover Mapping and Estimation Yearly 30 m - [glance30v001](https://doi.org/10.5067/MEaSUREs/GLanCE/GLanCE30.001)
 - Science Dataset (SDS) layers:  
        - LC (Integer identifier for class in the current year)  
        - PrevClass	(Integer identifier for class in previous year, if change has occurred; N/A if no change)
 - The SDS Layers in this tutorial utilize a seven class integer system to identify land cover. The table below denotes the class number and its corresponding description.

|Value    |           Name               | Description  |
| :----:  |          :----:              | :---       |
| 1       |           Water              | Areas covered with water throughout the year: streams, canals, lakes, reservoirs, and oceans.      |
| 2       |          Ice/Snow            | Land areas where snow and ice cover is greater than 50% throughout the year.       |
| 3       |          Developed           | Areas of intensive use; land covered with structures, including any land functionally related to          developed/built-up activity.             |
| 4       | Barren/Sparsely Vegetated    | Land consists of natural occurrences of soils, sand, or rocks where less than 10% of the area is vegetated.       |
| 5       |          Tree Cover          | Land where the tree cover is greater than 30%. Note that cleared trees (i.e., clear-cuts) are mapped according to current cover (e.g., barren/sparsely vegetated, shrubs, or grasses).       |
| 6       |          Shrublands          |Land with less than 30% tree cover, where total vegetation cover exceeds 10% and shrub cover is greater than 10%.       |
| 7       |          Herbaceous          | Land covered by herbaceous cover. Total vegetation cover exceeds 10%, tree cover is less than 30%, and shrubs comprise less than 10% of the area.      |

## Tutorial Outline  

1. [**Getting Started**](#getstarted)  
    1.1 Import Packages<br>
    1.2 Setup Current Working Directory  
    1.3 EarthData Login  
2. [**Finding GLanCE30 Data**](#find)<br>
    2.1 Set spatial and temporal search parameters<br>
    2.2 Visualize where the spatial location is we are working with<br>
    2.3 Perform Data Query for GLanCE30 data within our geographic and temporal periods of interest and review the results   
3. [**Accessing GLanCE30 COG Data in the Cloud**](#cloudaccess)  
    3.1 Group by Band
4. [**Processing GLanCE30 Data**](#processglance)   
    4.1 Load GLanCE30 data into memory and subset spatially  
    4.2 Make all 36 subset SDS Layers into one xarray dataset 
    4.3 Export to COG  
5. [**Visualizations of GLanCE30 Data**](#Visualizations)
    5.1 Visualize Land Cover from the GLanCE30 Land Cover Time Series<br>
    5.2 Visualize Quantitative Land Cover for the Time Series in Line Chart Format<br>
    5.3 Visualize Quantitative Previous Class Time Series in Line Chart Format<br>
    5.4 Visualize Land Cover from the GLanCE30 Previous Class Time Series<br>
    5.5 Visualize Land Cover from the GLanCE30 Previous Class Time Series with a referance basemap<br>

## 1. Getting Started<a id="getstarted"></a>

### 1.1 Import Packages 

Import the required packages.

In [ ]:
import os
from datetime import datetime
from collections import defaultdict
import numpy as np
import pandas as pd
import geopandas as gp
from osgeo import gdal
import xarray as xr
import rioxarray as rxr
import hvplot.xarray
import earthaccess
import holoviews as hv
from holoviews import opts
import folium
from folium import Marker
from holoviews.plotting.util import process_cmap
from shapely.geometry import Polygon

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 1.2 Setup Current Working Directory

We will use a directory called data is the default working directory for the tutorial. Any required ancillary files will be read from here also any outputs downloaded will be written here.

In [ ]:
os.chdir('../../data')

### 1.3 Earthdata Login Authentication

We will use the [earthaccess](https://github.com/nsidc/earthaccess#readme) package for authentication. `earthaccess` can either create a new local .netrc file to store credentials or validate that one exists already in you user profile. If you do not have a .netrc file, you will be prompted for your credentials and one will be created. 

In [ ]:
earthaccess.login(persist=True)

## 2. Finding GLanCE30 Data using `earthaccess` <a id="find"></a>

To find GLanCE30 data, we will use the `earthaccess` python library to search NASA's Common Metadata Repository (CMR) for GLanCE30 data. We will use a geojson file containing our region of interest (ROI) to search for files that intersect. To do this, we will simplify it to a bounding box. Grab the bounding coordinates from the geopandas object after opening.

### 2.1 Set spatial and temporal search parameters

First we will read in our geojson file using geopandas. We will use the `total_bounds` property to get the bounding box of our ROI, and add that to a python tuple, which is the expected data type for the bounding_box parameter `earthaccess` `search_data`.

In [ ]:
field = gp.read_file('waubay_nwf.geojson')
bbox = tuple(list(field.total_bounds))
bbox

When searching we can also search a specific time period of interest. Here we search from the beginning of January 2002 to the end of December 2019. GLanCE30 is an annual product where each year is specified with the data of July 1st of each year. Therefore, we would expect to have 18 GLanCE30 products in the time series.   

In [ ]:
temporal = ("2002-01-01T00:00:00", "2019-12-31T23:59:59")

## 2.2 Perform query for GLanCE30 data and review results

Here we perform the query to CMR for GLanCE30 data that meets our specified spatial and temporal areas of interest. 

In [ ]:
results = earthaccess.search_data(
    short_name=['GLanCE30'],
    bounding_box=bbox,
    temporal=temporal
)

We can preview these results in a `pandas` `dataframe` we want to check the metadata. Note there are 18 results returned.

In [ ]:
pd.json_normalize(results)

We can also preview each individual results by selecting it from the list. This will show the data links, and a browse image. Note the seven different  SDS layers available from the GLanCE30 product. For this tutorial we will be working with the LC and PrevClass SDS Layers

In [ ]:
results[0]

We can also examine the umm metadata which is returned as a python dictionary. In our example here we will extract the spatial bounds of one GLanCE30 product and plot the bounds on a map. 

In [ ]:
metadata = results[0]["umm"]
metadata

Here, we are going to extract the geometry from one GLanCE30 product. Do do this we need to key the geometry of the product from umm metadata and covert it to a polygon using the `shapely` library.

In [ ]:
glance_geometry = results[0]['umm']['SpatialExtent']['HorizontalSpatialDomain']['Geometry']
glance_points = glance_geometry['GPolygons'][0]['Boundary']['Points']
        # Create shapely geometry from polygons
glance_polygon = Polygon([[p["Longitude"], p["Latitude"]] for p in glance_points])

### 2.3 Visualize ROI

We can plot the spatial location geometry on a map visualization using the `folium` library. The geometry drawn in red shows the full extent of the GLanCE30 data. The geometry drawn in blue on the map represents the final extent we are be working with in the final xarray dataset. This visualization shows the exact amount of data we will process (blue) which is much smaller than the total of extent of the GLanCE30 product (red).

In [ ]:
m = folium.Map([46.0732, -97.7783], zoom_start=6, width=900, height=350, tiles='OpenStreetMap')
folium.GeoJson(field).add_to(m)
folium.GeoJson(glance_polygon, color = 'red').add_to(m)
m.add_child(Marker(location=[45.3676, -97.4048], popup='Waubay, South Dakota AOI', icon = folium.Icon(color = 'red')))
folium.LatLngPopup().add_to(m)
m

## 3. Accessing GLanCE30 Cloud Optimized GeoTIFFs (COGs) from Earthdata Cloud <a id="cloudaccess"></a>
Now that we have a list of data URLs, we will configure `gdal` and `rioxarray` to access the cloud assets that we are interested in, and read them directly into memory without needing to download the files. 

The Python libraries used to access COG files in Earthdata Cloud leverage GDAL's virtual file systems. Whether you are running this code in the Cloud or in a local workspace, GDAL configurations must be set in order to successfully access the GLanCE30 COG files. The settings below enable GDAL to send authentication information when accessing the GLanCE30 COG files in the Earthdata Cloud and also enable retrying connections in case of network issues.

In [ ]:
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')
gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL', 'YES')
gdal.SetConfigOption('GDAL_HTTP_MAX_RETRY', '10')
gdal.SetConfigOption('GDAL_HTTP_RETRY_DELAY', '0.5')

### 3.1 Group by Layer Type

Here we can see a list of all the URLs for each GLanCE30 product. Since we have 18 years in our time series and there are seven SDS layers in each year we have here 126 URLs in this list. 

In [ ]:
glance_results_urls = [granule.data_links() for granule in results]
glance_results_urls

Here we group the urls by the SDS Layer Type and load this into a python dictionary.

In [ ]:
def varname_from_url(url):
    return url.split("/")[-1].split(".")[0].split("_")[-1]

grouped_urls = defaultdict(list)
for granule in glance_results_urls:
    for url in granule:
        var = varname_from_url(url)
        grouped_urls[var].append(url)

results_dict = dict(grouped_urls)
results_dict

## 4. Processing GLanCE30 Data <a id="processglance"></a>

In this section we will load the SDS Layers for LC and PrevClass from the results_dict dictionary created above by lazy loading the SDS Layers an xarray.  We will also subset each SDS layer to the geojson area of interest used to make the query for the GLanCE30. The result will be a image of the SDS layers that are clipped to the geojson area of interest.

### 4.1 Load GLanCE30 data into memory and subset spatially

Here we create a function to call when lazy loading the SDS Layers using `rioxarray`. The urls for the SDS Layers are stored in the results_dict created above. This function will handle adding the date of the SDS layer as a dimension to the xarray dataset a well as clipping the dataset to the geojson area of interest file.

In [ ]:
def preprocess(ds):
    # Squeeze Band
    ds = ds.squeeze('band', drop=True)
    # Time Dimension
    url = ds.encoding['source']
    print('Loading {}'.format(url))
    date_str = url.split('/')[-1].split('_')[1].split('A')[1]
    date = datetime.strptime(date_str, '%Y%m%d')
    ds = ds.assign_coords(date=date)
    # Clip
    field_reproj = field.to_crs(ds.rio.crs)
    clipped = ds.rio.clip(field_reproj.geometry.values, field_reproj.crs, all_touched=True)
    return clipped

Here we handle the loading the SDS Layers. Of the available layers we are only loading the layers we want the LC and PrevClass SDS Layers. Each SDS Layer contributes 18 products to the final xarray.

In [ ]:
ds_lc = xr.open_mfdataset(results_dict['LC'],
                       engine='rasterio',
                       band_as_variable=False,
                       combine='nested',
                       concat_dim='date',
                       preprocess=preprocess,
                       coords='minimal',
                       ).rename({"band_data":'LC'})
ds_prev = xr.open_mfdataset(results_dict['PrevClass'],
                       engine='rasterio',
                       band_as_variable=False,
                       combine='nested',
                       concat_dim='date',
                       preprocess=preprocess,
                       coords='minimal',
    ).rename({"band_data":"PrevClass"})

### 4.2 Make all 36 subset SDS Layers into one xarray dataset

Here we make the one xarray dataset containing all 36 of the subset SDS Layers created above by running a concat merge command on the two datasets created above. We can also see the size of the final array and view its contents.

In [ ]:
glance_dataset = xr.merge([ds_lc,ds_prev]).load()
print('Dataset Size (Gb): ', glance_dataset.nbytes/1e9)
glance_dataset

## 5. Visualizations of GLanCE30 Data <a id="visualize"></a>

In this section we will explorer different ways to gain information from the subset LC and PrevClass GLanCE30 SDS layers. This will leverage the glance_dataset xarray dataset created in section 4 with various filtering techniques. We will make a map visualizations of the Land Cover classes across the time series, show the number of pixels of each Land Cover Class as a line chart, show where land cover has changed from the previous year using the PrevClass SDS layer and show the number pixels of each class that changed from the previous year as a line chart, finally we will show a map visualization of change in the time series overlayed on a reference map. This will be done using the `holoviews` python module.

### 5.1 Visualize Land Cover from the GLanCE30 Land Cover Time Series

Here, we will make a custom color ramp to display the Land Cover time series with the classes represented as a the following colors. 

- 1 = Water (Blue)
- 2 = Snow (White)
- 3 = Developed (Red)
- 4 = Barren (Grey)
- 5 = Tree Cover (Green)
- 6 = Shrub (Brown)
- 7 = Herbaceous (Tan)

In [ ]:
rgbs = [(70,107,159),(209,222,248),(171,0,0),(179,172,159),(181,197,143),(204,184,121),(223,223,194)]
color_list = ['#{:02x}{:02x}{:02x}'.format(r, g, b) for r, g, b in rgbs]

Here we will make a visualization of the Land Cover Time Series. Cycle through the time series by using the slider bar in the visualization.

In [ ]:
land_cover_img = glance_dataset['LC'].hvplot('x', 'y', groupby='date', dynamic=True, rasterize=True, width=700, height=500, cmap = color_list)
land_cover_img

### 5.2 Visualize Quantitative Land Cover for the Time Series in Line Chart Format

In this section we will construct a line chart showing the number of pixels for each Land Cover class by year. To start we will need to filter the glance_dataset xarray LC_TIF coordinate by the appropriate  land cover type and make the others as nodata and assign that to new variable. This will then be constructed as a `pandas` dataframe so we can visualize the land cover types counts in a tabular format.

In [ ]:
water_pixels = glance_dataset['LC'].where(glance_dataset['LC'] == 1, np.nan).count(('x', 'y'))
ice_snow_pixels = glance_dataset['LC'].where(glance_dataset['LC'] == 2, np.nan).count(('x', 'y'))
developed_pixels = glance_dataset['LC'].where(glance_dataset['LC'] == 3, np.nan).count(('x', 'y'))
barren_sparsely_vegetated_pixels = glance_dataset['LC'].where(glance_dataset['LC'] == 4, np.nan).count(('x', 'y'))
tree_cover_pixels = glance_dataset['LC'].where(glance_dataset['LC'] == 5, np.nan).count(('x', 'y'))
shrublands_pixels = glance_dataset['LC'].where(glance_dataset['LC'] == 6, np.nan).count(('x', 'y'))
herbaceous_pixels = glance_dataset['LC'].where(glance_dataset['LC'] == 7, np.nan).count(('x', 'y'))

In [ ]:
land_cover_data = {'date':water_pixels['date'].dt.year,'Water':water_pixels, 'Snow_Ice':ice_snow_pixels, 'Developed': developed_pixels, 'Barren': barren_sparsely_vegetated_pixels,
       'Tree Cover':tree_cover_pixels, 'Shrublands':shrublands_pixels, 'Herbaceous':herbaceous_pixels}

In [ ]:
land_cover_table = pd.DataFrame(land_cover_data)
land_cover_table

Now we can take the tabular data and create a line chart showing the counts of each land cover over the time series. We can construct the chart using the same custom color ramp we used for the map visualization. 

In [ ]:
chart_colors = {'Water':color_list[0], 'Snow_Ice':color_list[1], 'Developed':color_list[2], 'Barren':color_list[3],
                'Tree Cover':color_list[4],'Shrublands':color_list[5],'Herbaceous':color_list[6]}

In [ ]:
land_cover_counts = {}
for column in land_cover_table.columns:
    if column != 'date':
        land_cover_counts[column] = hv.Curve((land_cover_table['date'], land_cover_table[column]), label=column).opts(
        opts.Curve(color=chart_colors[column]))

In [ ]:
landcover_plot = hv.Overlay(land_cover_counts).opts(
    height=300, 
    width=600,
    xlabel='Year', 
    ylabel='Number of Pixels', 
    title='Land Cover Class Counts from 2002 - 2019',
    legend_position='right',
    yformatter='%.0f')
landcover_plot

### 5.3 Visualize Quantitative Previous Class Time Series in Line Chart Format

In this section we will construct a line chart showing the number of pixels that changed for each Land Cover class by year. To start we will need to filter the glance_dataset xarray Prev_TIFS coordinate by the appropriate land cover type and make the others as nodata and assign that to new variable. This will then be constructed as a `pandas` dataframe so we can visualize the land cover change class counts in a tabular format.

In [ ]:
water_change_pixels = glance_dataset['PrevClass'].where(glance_dataset['PrevClass'] == 1, np.nan).count(('x', 'y'))
ice_snow__change_pixels = glance_dataset['PrevClass'].where(glance_dataset['PrevClass'] == 2, np.nan).count(('x', 'y'))
developed_change_pixels = glance_dataset['PrevClass'].where(glance_dataset['PrevClass'] == 3, np.nan).count(('x', 'y'))
barren_sparsely_vegetated__change_pixels = glance_dataset['PrevClass'].where(glance_dataset['PrevClass'] == 4, np.nan).count(('x', 'y'))
tree_cover_change_pixels = glance_dataset['PrevClass'].where(glance_dataset['PrevClass'] == 5, np.nan).count(('x', 'y'))
shrublands_change_pixels = glance_dataset['PrevClass'].where(glance_dataset['PrevClass'] == 6, np.nan).count(('x', 'y'))
herbaceous_change_pixels = glance_dataset['PrevClass'].where(glance_dataset['PrevClass'] == 7, np.nan).count(('x', 'y'))

In [ ]:
land_cover_change_data = {'date':water_change_pixels['date'].dt.year,'Water':water_change_pixels, 'Snow_Ice':ice_snow__change_pixels, 
                          'Developed': developed_change_pixels, 'Barren': barren_sparsely_vegetated__change_pixels, 'Tree Cover':tree_cover_change_pixels,
                          'Shrublands':shrublands_change_pixels, 'Herbaceous':herbaceous_change_pixels}

In [ ]:
land_cover_change_table = pd.DataFrame(land_cover_change_data)
land_cover_change_table

Now we can take the tabular data and create a line chart showing the counts of each land cover over the time series. The chart can be constructed using the same color ramp created for the chart in section 5.3.

In [ ]:
land_cover_change_counts = {}
for column in land_cover_change_table.columns:
    if column != 'date':
        land_cover_change_counts[column] = hv.Curve((land_cover_change_table['date'], land_cover_change_table[column]), label=column).opts(
        opts.Curve(color=chart_colors[column]))

In [ ]:
land_cover_change_plot = hv.Overlay(land_cover_change_counts).opts(
    height=300, 
    width=600,
    xlabel='Year', 
    ylabel='Number of Pixels', 
    title='Previous Year Land Cover Class Change Counts from 2002 - 2019',
    legend_position='right',
    yformatter='%.0f')
land_cover_change_plot

### 5.4 Visualize Land Cover from the GLanCE30 Previous Class Time Series

Here we will make a visualization of the Previous Land Cover Time Series. This will show pixels that have a different land cover class in the current year then it did in the previous year by having a class value other than nodata. Cycle through the time series by using the slider bar in the visualization.

In [ ]:
land_cover_change_pixels = glance_dataset.where((~np.isnan(glance_dataset['PrevClass'])))

In [ ]:
change_img = land_cover_change_pixels.hvplot('x', 'y', groupby='date', dynamic=True, rasterize=True, width=700, height=500, cmap = color_list)
change_img

### 5.5 Visualize Land Cover from the GLanCE30 Previous Class Time Series with a referance basemap

Next we will overlay the areas of change from the time series on a basemap to aid in our spatial awareness. To do this first, we will need to reproject the xarray dataset created by in section 5.4 to a common CRS of WGS84 in order to see the basemap.

In [ ]:
change_reprojected_dataset = land_cover_change_pixels.rio.reproject('epsg:4326')

In [ ]:
change_img_reprojected = change_reprojected_dataset.hvplot('x', 'y', groupby='date', crs = 'epsg:4326', tiles=True, dynamic=True, rasterize=True, width=700, height=500, cmap = color_list)
change_img_reprojected

Success! You have learned how to query for and visualize GLanCE30 Land Cover Data without downloading any of the source data. You can now replace the geojson file, and temporal range in section 2 with your own and re-run the notebook.

## Contact Info  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://www.earthdata.nasa.gov/centers/lp-daac>  
Date last modified: 06-04-2025  

¹Work performed under USGS contract G15PD00467 for NASA contract NNG14HH33I.